In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./file/4.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =


work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1
                
                try:
                    resp = requests.get(f'''https://www.auveco.com/store_ss2/extensions/UnlockCommerce/YearMakeModel/1.0.0/Modules/YearMakeModel/SuiteScript2/YearMakeModel.Service.ss?action=listbyymm&limit=100&make={input_.loc[a, 'Make']}&model={input_.loc[a, 'Model']}&offset={input_.loc[a, 'Page']*100-100}&page={input_.loc[a, 'Page']}&sort=relevance%3Adesc&year={input_.loc[a, 'Year']}''',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        json_ = resp.json()

                        list_item = [str(dict_['id']) for dict_ in json_['itemsId']]
                        if json_['success'] == 'OK' and list_item:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'Make': input_.loc[a, 'Make'],
                                     'Model': input_.loc[a, 'Model'],
                                     'Year': input_.loc[a, 'Year'],
                                     'SKU Count': input_.loc[a, 'SKU Count'],
                                     'Page': input_.loc[a, 'Page'],
                                     'Item': '%2C'.join(list_item)}])
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Make']} > {input_.loc[a, 'Model']} > {input_.loc[a, 'Year']} > Page {input_.loc[a, 'Page']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make', 'Model', 'Year', 'Page'],
                                                ascending=[True, True, False, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/5.item-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Make', 'Model', 'Year', 'Page'],
                                            ascending=[True, True, False, True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/item_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：6] - TOYOTA > Solara > 2001 > Page 1
[剩余数量：1000] - [当前时间：19:01:13]

[状态：ok，尝试次数：1] - TOYOTA > Starlet > 1981 > Page 1
[剩余数量：999] - [当前时间：19:01:13]

[状态：ok，尝试次数：31] - TOYOTA > Solara > 2008 > Page 1
[剩余数量：998] - [当前时间：19:01:13]

[状态：ok，尝试次数：1] - TOYOTA > Solara > 1999 > Page 1
[剩余数量：997] - [当前时间：19:01:13]

[状态：ok，尝试次数：9] - TOYOTA > Starlet > 1984 > Page 1
[剩余数量：996] - [当前时间：19:01:13]

[状态：ok，尝试次数：3] - TOYOTA > Starlet > 1983 > Page 1
[剩余数量：995] - [当前时间：19:01:13]

[状态：ok，尝试次数：2] - TOYOTA > Supra > 1998 > Page 1
[剩余数量：994] - [当前时间：19:01:14]

[状态：ok，尝试次数：5] - TOYOTA > Starlet > 1980 > Page 1
[剩余数量：993] - [当前时间：19:01:15]

[状态：ok，尝试次数：4] - TOYOTA > Supra > 1996 > Page 1
[剩余数量：992] - [当前时间：19:01:15]

[状态：ok，尝试次数：1] - TOYOTA > Supra > 1991 > Page 1
[剩余数量：991] - [当前时间：19:01:15]

[状态：ok，尝试次数：11] - TOYOTA > Supra > 1997 > Page 1
[剩余数量：990] - [当前时间：19:01:15]

[状态：ok，尝试次数：5] - TOYOTA > Supra > 1988 > Page 1
[剩余数量：989] - [当前时间：19:01:16]

[状态：ok，尝试次数：6] - TOYOTA > Supra > 1993 > Page 1
[剩